# Singleton

싱글톤 디자인 패턴
- 글로벌하게 접근 가능한 하나의 객체를 제공하는 패턴
- 동일한 리소스에 대한 **동시 요청의 충돌을 방지**하기 위해 하나의 인스턴스를 공유하는 작업에 주로 사용
  - e.g. 데이터의 일관성 유지를 위해 DB에 작업을 수행하는 하나의 데이터 베이스 객체가 필요한 경우


In [1]:
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance

In [2]:
s = Singleton()
print("Object created", s, id(s))

s1 = Singleton()
print("Object created", s1, id(s1))

Object created <__main__.Singleton object at 0x000001ED51ED13C8> 2118793368520
Object created <__main__.Singleton object at 0x000001ED51ED13C8> 2118793368520


#### lazy instantiation

위 예제와 다르게, 인스턴스가 꼭 필요한 시점에 생성

In [3]:
class Singleton:
    __instance = None
    def __init__(self):
        if not Singleton.__instance:
            print("__init__ method called...")
        else:
            print("Instance already created:", self.getInstance())
    
    @classmethod
    def getInstance(cls):
        if not cls.__instance:
            cls.__instance = Singleton()
        return cls.__instance

In [4]:
s = Singleton()
print("Object created", Singleton.getInstance())
s1 = Singleton()
print("Object created", s1)

__init__ method called...
__init__ method called...
Object created <__main__.Singleton object at 0x000001ED53A26D30>
Instance already created: <__main__.Singleton object at 0x000001ED53A26D30>
Object created <__main__.Singleton object at 0x000001ED53A26A58>


# Monostate Singleton

모든 객체가 같은 상태를 공유하는 패턴

In [10]:
class Borg:
    __shared_state = {"1": "2"}
    def __init__(self):
        self.x = 1
        self.__dict__ = self.__shared_state
        pass

In [11]:
b = Borg()
b1 = Borg()
b.x = 4

print("Borg Object 'b': ", b)
print("Borg Object 'b1': ", b1)
print("Object State 'b':", b.__dict__)
print("Object State 'b1':", b1.__dict__)

Borg Object 'b':  <__main__.Borg object at 0x000001ED52062780>
Borg Object 'b1':  <__main__.Borg object at 0x000001ED5358B400>
Object State 'b': {'1': '2', 'x': 4}
Object State 'b1': {'1': '2', 'x': 4}


In [8]:
class Borg(object):
    _shared_state = {}
    def __new__(cls, *args, **kwargs):
        obj = super(Borg, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_state
        return obj

In [9]:
b = Borg()
b1 = Borg()
b.x = 4

print("Borg Object 'b': ", b)
print("Borg Object 'b1': ", b1)
print("Object State 'b':", b.__dict__)
print("Object State 'b1':", b1.__dict__)

Borg Object 'b':  <__main__.Borg object at 0x000001ED53C7ADD8>
Borg Object 'b1':  <__main__.Borg object at 0x000001ED53C7ADA0>
Object State 'b': {'x': 4}
Object State 'b1': {'x': 4}


## MetaClass (MetaKls)

In [14]:
# metaclass 구현
class MyInt(type):
    def __call__(cls, *args, **kwargs):
        print("***** Here's My int *****", args)
        print("Now do whatever you want with these objects...")
        return type.__call__(cls, *args, **kwargs)
    
class int(metaclass=MyInt):
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [15]:
i = int(4, 5)

***** Here's My int ***** (4, 5)
Now do whatever you want with these objects...


In [16]:
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]
    
class Logger(metaclass=MetaSingleton):
    pass

In [17]:
logger1 = Logger()
logger2 = Logger()
print(logger1, logger2)

<__main__.Logger object at 0x000001ED5381BBE0> <__main__.Logger object at 0x000001ED5381BBE0>


### 실습 1

In [19]:
import sqlite3
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]
    
class Database(metaclass=MetaSingleton):
    connection = None
    def connect(self):
        if self.connection is None:
            self.connection = sqlite3.connect("db.sqlite3")
            self.cursorobj = self.connection.cursor()
        return self.cursorobj
    
db1 = Database().connect()
db2 = Database().connect()

print("Database Objects DB1", db1)
print("Database Objects DB2", db2)

Database Objects DB1 <sqlite3.Cursor object at 0x000001ED537B1E30>
Database Objects DB2 <sqlite3.Cursor object at 0x000001ED537B1E30>


In [24]:
MetaSingleton._instances

{__main__.Database: <__main__.Database at 0x1ed53a9cd68>,
 __main__.Database2: <__main__.Database2 at 0x1ed51fb5400>}

### 실습2

In [29]:
class HealthCheck:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not HealthCheck._instance:
            HealthCheck._instance = super(HealthCheck, cls).__new__(cls, *args, **kwargs)
        return HealthCheck._instance
    
    def __init__(self):
        self._servers = []
    
    def addServer(self):
        self._servers.append("Server 1")
        self._servers.append("Server 2")
        self._servers.append("Server 3")
        self._servers.append("Server 4")
    
    def changeServer(self):
        self._servers.pop()
        self._servers.append("Server 5")

In [30]:
hc1 = HealthCheck()
hc2 = HealthCheck()

hc1.addServer()
print("Schedule health check for servers (1)..")
for i in range(4):
    print("Checking ", hc1._servers[i])

hc2.changeServer()
print("Schedule health check for servers (2)..")
for i in range(4):
    print("Checking ", hc2._servers[i])

Schedule health check for servers (1)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 4
Schedule health check for servers (2)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 5
